In [1]:
!pip install pytorch-lightning

In [2]:
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
from src.tools import *

In [4]:
class SamenessModule(nn.Module):
    def __init__(self, layer_sizes=(1, 8, 8, 8, 1), activation_fun=nn.ReLU()):
        super().__init__()

        self.run_counter = 0

        layers_list = []
        for i in range(len(layer_sizes) - 1):
            layers_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i != len(layer_sizes) - 2:
                layers_list.append(activation_fun)

        self.l1 = nn.Sequential(*layers_list)
        print(self)

        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        self.run_counter += 1
        return self.l1(x.double())


class SamenessAutoEncoder(pl.LightningModule):
    def __init__(self, encoder: SamenessModule):
        super().__init__()
        self.encoder = encoder

    def training_step(self, batch, batch_idx):
        batch = batch[:, 0, :]
        x, y = batch.reshape(2, len(batch), 1)
        x_hat = self.encoder(x)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5 * 1e-3)
        return optimizer

In [5]:
class SamenessBinModule(nn.Module):
    def __init__(self, layer_sizes=(64, 8, 8, 8, 64), activation_fun=nn.ReLU()):
        super().__init__()

        self.run_counter = 0

        layers_list = []
        for i in range(len(layer_sizes) - 1):
            layers_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i != len(layer_sizes) - 2:
                layers_list.append(activation_fun)

        self.l1 = nn.Sequential(*layers_list)
        print(self)

        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        self.run_counter += 1
        return self.l1(x.double())


class SamenessBinAutoEncoder(pl.LightningModule):
    def __init__(self, encoder: SamenessModule):
        super().__init__()
        self.encoder = encoder

    def training_step(self, batch, batch_idx):
        batch = batch[:, 0, :]
        x, y = batch.reshape(2, len(batch), 64)
        x_hat = self.encoder(x)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5 * 1e-3)
        return optimizer

In [6]:
from torch.utils.data import Dataset
import torch
import os
import torch.random
import pandas as pd

In [7]:
from notebooks.data.SimpleRandomDataset import SimpleRandomDataset
from notebooks.data.SimpleRandomDataset import SimpleRandomBitDataset

In [8]:
# Parameters:
activation_fun = nn.ReLU()
layer_sizes = [1, 8, 8, 8, 1]
# layer_sizes = [1, 32, 32, 32, 1]

#### Training with:
n = 50000 batch_size = 5000 max_epochs=100 -> takes 102 sec
n = 50000 batch_size = 2500 max_epochs=100 -> takes 66 sec
n = 50000 batch_size = 1000 max_epochs=100 -> takes 81 sec
n = 50000 batch_size = 500 max_epochs=100 -> takes 107 sec

In [9]:
# dataset = SimpleRandomDataset(50000, transform=transforms.ToTensor())
# train_loader = DataLoader(dataset, batch_size=2500,
#                           # num_workers=8
#                           )
#
# # model
# autoencoder = SamenessAutoEncoder(SamenessModule(layer_sizes, activation_fun))

In [10]:
dataset = SimpleRandomBitDataset(50000, transform=transforms.ToTensor())
train_loader = DataLoader(dataset, batch_size=2500,
                          # num_workers=8
                          )

# model
autoencoder = SamenessBinAutoEncoder(SamenessBinModule())

SamenessBinModule(
  (l1): Sequential(
    (0): Linear(in_features=64, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=64, bias=True)
  )
)


In [11]:
# train model
trainer = pl.Trainer(max_epochs=100)
# trainer.fit(model=autoencoder, train_dataloaders=train_loader)
with Timer():
    trainer.fit(autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, 

Running with 


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params
----------------------------------------------
0 | encoder | SamenessBinModule | 1.2 K 
----------------------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)
C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_

Epoch 99: 100%|██████████| 20/20 [00:00<00:00, 27.08it/s, v_num=130]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 20/20 [00:00<00:00, 26.61it/s, v_num=130]
Took 78.3051 seconds


In [12]:
from tqdm import tqdm

In [ ]:
# test_dataset = SimpleRandomDataset(10000, transform=transforms.ToTensor())
# test_loader = DataLoader(test_dataset, batch_size=10)

In [14]:
test_dataset = SimpleRandomBitDataset(10000, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=10)

In [19]:

sum_dif = 0
with torch.no_grad():
    # for batch in tqdm(test_loader):
    for index, batch in tqdm(enumerate(test_loader)):
        batch = batch[:, 0, :]
        x, y = batch.reshape(2, len(batch), 64)
        reconstructed_x = autoencoder.encoder(x)
        if index == 0:
            print("Original x:", x)
            print("Reconstructed x:", reconstructed_x)
        sum_dif += ((x - reconstructed_x) ** 2).sum()
print(f"square sum: {(sum_dif / (index + 1) / 10)}")

428it [00:00, 2150.55it/s]

Original x: tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
         1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
         1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
         1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0.,

1000it [00:00, 2369.72it/s]

square sum: 12.45458891937482


In [20]:
n = 1000
s = 0
c = 0
with torch.no_grad():
    for val in test_dataset:
        c += 1
        s += autoencoder.encoder(val)
print(f"avarage diff: {s / c}")

avarage diff: tensor([[[0.5047, 0.4838, 0.4775, 0.4750, 0.4558, 0.3975, 0.2669, 0.0967,
          0.5049, 0.4991, 0.5016, 0.4813, 0.4874, 0.5111, 0.4987, 0.4918,
          0.4862, 0.5045, 0.4816, 0.5010, 0.4994, 0.4879, 0.4941, 0.5006,
          0.5037, 0.5159, 0.5085, 0.4939, 0.5022, 0.5038, 0.5070, 0.5141,
          0.4943, 0.4988, 0.4997, 0.5027, 0.5013, 0.4889, 0.4952, 0.4986,
          0.5065, 0.5022, 0.5127, 0.4830, 0.4962, 0.4991, 0.4976, 0.5085,
          0.3926, 0.3702, 0.3188, 0.4595, 0.4139, 0.4162, 0.4871, 0.5164,
          0.5067, 0.6048, 0.3930, 0.3930, 0.3915, 0.3924, 0.3932, 0.3934],
         [0.5047, 0.4838, 0.4775, 0.4750, 0.4558, 0.3975, 0.2669, 0.0967,
          0.5049, 0.4991, 0.5016, 0.4813, 0.4874, 0.5111, 0.4987, 0.4918,
          0.4862, 0.5045, 0.4816, 0.5010, 0.4994, 0.4879, 0.4941, 0.5006,
          0.5037, 0.5159, 0.5085, 0.4939, 0.5022, 0.5038, 0.5070, 0.5141,
          0.4943, 0.4988, 0.4997, 0.5027, 0.5013, 0.4889, 0.4952, 0.4986,
          0.5065, 0.502

In [39]:
x = np.array([FloatBitsConverter.get_float_bits_in_memory(val) for val in np.linspace(-20, 20, 1000)])
x = x.reshape(len(x), 64)
x = torch.tensor(x)

y = autoencoder.encoder(x)

x = x.detach().numpy()
y = y.detach().numpy()
differ = (x - y) ** 2
print(differ.sum(axis=1))
x_to_plot = [FloatBitsConverter.get_float_by_bits(val) for val in x]
y_to_plot = [FloatBitsConverter.get_float_by_bits(val) for val in y]

[ 6.18952094 12.50849059 12.49988632 13.60086661 12.68280303 13.13276382
 13.47370979 13.94199964 10.91678816 13.63886311 13.58762193 13.00342108
 13.71402115 13.34830251 14.01550211 13.11030722 13.12181214 14.29586132
 13.44550694 13.71201294 13.32792327 14.40202065 13.68470565 13.63446231
 13.30340055 11.82132259 13.87238047 13.50282717 13.40193377 13.84683518
 13.40867496 13.0895277  12.37939996 13.58426898 13.8692066  13.97728844
 13.84470738 14.06903426 13.71659916 13.57320883 11.93109205 14.26792238
 14.074694   13.64934883 13.74336486 14.0437329  13.28289861 13.69403844
 13.50741892 13.7785833  11.17991583 13.84456015 12.9943631  14.04368443
 13.25501063 12.91429521 12.79135191 13.55188184 13.4404203  13.409558
 13.54430821 13.88394897 13.16978689 13.71391739 12.07089082 14.08905384
 14.13642369 13.86941521 13.80619233 14.2322946  13.51841465 13.86620481
 13.30596176 13.8360708  13.78298514 13.35816392 13.70747859 13.90081425
 12.41719587 13.43442288 13.29468487 13.29565869 13.6

In [32]:
import plotly.express as px

In [60]:
print(x[0].astype(int))
# print(y[0])
print(y[0].round().astype(int))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [52]:
# print(x_to_plot)
print(y_to_plot)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [46]:
fig = px.line(x=x_to_plot, y=y_to_plot)
fig.show()

fig = px.line(x=x_to_plot, y=differ.sum(axis=1))
fig.show()